In [ ]:
def read_data():
    print('Reading train.csv file....')
    train = pd.read_csv('/kaggle/input/data-science-bowl-2019/train.csv')
    print('Training.csv file have {} rows and {} columns'.format(train.shape[0], train.shape[1]))

    print('Reading test.csv file....')
    test = pd.read_csv('/kaggle/input/data-science-bowl-2019/test.csv')
    print('Test.csv file have {} rows and {} columns'.format(test.shape[0], test.shape[1]))

    print('Reading train_labels.csv file....')
    train_labels = pd.read_csv('/kaggle/input/data-science-bowl-2019/train_labels.csv')
    print('Train_labels.csv file have {} rows and {} columns'.format(train_labels.shape[0], train_labels.shape[1]))

    print('Reading specs.csv file....')
    specs = pd.read_csv('/kaggle/input/data-science-bowl-2019/specs.csv')
    print('Specs.csv file have {} rows and {} columns'.format(specs.shape[0], specs.shape[1]))

    print('Reading sample_submission.csv file....')
    sample_submission = pd.read_csv('/kaggle/input/data-science-bowl-2019/sample_submission.csv')
    print('Sample_submission.csv file have {} rows and {} columns'.format(sample_submission.shape[0], sample_submission.shape[1]))
    return train, test, train_labels, specs, sample_submission

def encode_title(train, test, train_labels):
    # encode title
    train['title_event_code'] = list(map(lambda x, y: str(x) + '_' + str(y), train['title'], train['event_code']))
    test['title_event_code'] = list(map(lambda x, y: str(x) + '_' + str(y), test['title'], test['event_code']))
    all_title_event_code = list(set(train["title_event_code"].unique()).union(test["title_event_code"].unique()))
    # make a list with all the unique 'titles' from the train and test set
    list_of_user_activities = list(set(train['title'].unique()).union(set(test['title'].unique())))
    # make a list with all the unique 'event_code' from the train and test set
    list_of_event_code = list(set(train['event_code'].unique()).union(set(test['event_code'].unique())))
    list_of_event_id = list(set(train['event_id'].unique()).union(set(test['event_id'].unique())))
    # make a list with all the unique worlds from the train and test set
    list_of_worlds = list(set(train['world'].unique()).union(set(test['world'].unique())))
    # create a dictionary numerating the titles
    activities_map = dict(zip(list_of_user_activities, np.arange(len(list_of_user_activities))))
    activities_labels = dict(zip(np.arange(len(list_of_user_activities)), list_of_user_activities))
    activities_world = dict(zip(list_of_worlds, np.arange(len(list_of_worlds))))
    assess_titles = list(set(train[train['type'] == 'Assessment']['title'].value_counts().index).union(set(test[test['type'] == 'Assessment']['title'].value_counts().index)))
    # replace the text titles with the number titles from the dict
    train['title'] = train['title'].map(activities_map)
    test['title'] = test['title'].map(activities_map)
    train['world'] = train['world'].map(activities_world)
    test['world'] = test['world'].map(activities_world)
    train_labels['title'] = train_labels['title'].map(activities_map)
    win_code = dict(zip(activities_map.values(), (4100*np.ones(len(activities_map))).astype('int')))
    # then, it set one element, the 'Bird Measurer (Assessment)' as 4110, 10 more than the rest
    win_code[activities_map['Bird Measurer (Assessment)']] = 4110
    # convert text into datetime
    train['timestamp'] = pd.to_datetime(train['timestamp'])
    test['timestamp'] = pd.to_datetime(test['timestamp'])
    
    
    return train, test, train_labels, win_code, list_of_user_activities, list_of_event_code, activities_labels, assess_titles, list_of_event_id, all_title_event_code


# read data
train, test, train_labels, specs, sample_submission = read_data()
# get usefull dict with maping encode
train, test, train_labels, win_code, list_of_user_activities, list_of_event_code, activities_labels, assess_titles, list_of_event_id, all_title_event_code = encode_title(train, test, train_labels)
# tranform function to get the train and test set

In [ ]:
def get_train_and_test(train, test):
    compiled_train = []
    compiled_test = []
    compiled_val=[]
    for i, (ins_id, user_sample) in tqdm(enumerate(train.groupby('installation_id', sort = False)), total = 17000):
        compiled_train += get_data(user_sample)
    for ins_id, user_sample in tqdm(test.groupby('installation_id', sort = False), total = 1000):
        test_data,val_data = get_data(user_sample, test_set = True)
        compiled_test.append(test_data)
        compiled_val+=(val_data)
    reduce_train = pd.DataFrame(compiled_train)
    reduce_test = pd.DataFrame(compiled_test)
    reduce_val = pd.DataFrame(compiled_val)
    categoricals = ['session_title']
    return reduce_train, reduce_test,reduce_val

from tqdm import tqdm
import numpy as np
import pandas as pd
import os
import copy
import matplotlib.pyplot as plt
%matplotlib inline
# import catboost as cat
import time
from collections import Counter
import datetime
# from catboost import CatBoostRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, KFold, RepeatedKFold, GroupKFold, GridSearchCV, train_test_split, TimeSeriesSplit, RepeatedStratifiedKFold
from sklearn import metrics
from sklearn.metrics import classification_report, confusion_matrix
from sklearn import linear_model
import gc
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
# from bayes_opt import BayesianOptimization
# import eli5
# import shap
# from IPython.display import HTML
# import json
# import altair as alt
# from category_encoders.ordinal import OrdinalEncoder
# import networkx as nx
# import matplotlib.pyplot as plt
# %matplotlib inline
# from typing import List

# import os
# import time
# import datetime
# import json
# import gc
# from numba import jit

# import numpy as np
# import pandas as pd

# import matplotlib.pyplot as plt
# import seaborn as sns
# from tqdm import tqdm_notebook

# import lightgbm as lgb
# import xgboost as xgb
# from catboost import CatBoostRegressor, CatBoostClassifier
# from sklearn import metrics
# from typing import Any
# from itertools import product
# pd.set_option('max_rows', 500)
# import re
# from tqdm import tqdm
# from joblib import Parallel, delayed
from tqdm import tqdm_notebook
from sklearn.preprocessing import StandardScaler
from sklearn.svm import NuSVR, SVR
from sklearn.metrics import mean_absolute_error
pd.options.display.precision = 15
from collections import defaultdict
import lightgbm as lgb
import xgboost as xgb

def get_data(user_sample, test_set=False):
    '''
    The user_sample is a DataFrame from train or test where the only one 
    installation_id is filtered
    And the test_set parameter is related with the labels processing, that is only requered
    if test_set=False
    '''
    all_assessments=[]
    # Constants and parameters declaration
    event_code_count: Dict[str, int] = {ev: 0 for ev in list_of_event_code}
    event_id_count: Dict[str, int] = {eve: 0 for eve in list_of_event_id}
    title_count: Dict[str, int] = {eve: 0 for eve in activities_labels.values()} 
    title_event_code_count: Dict[str, int] = {t_eve: 0 for t_eve in all_title_event_code}
    
    
    
    ass_try_non_rec=0
    ass_true_non_rec=0
    ass_false_non_rec=0


    ass_try_rec=0
    ass_true_rec=0
    ass_false_rec=0
  
  
    try_non_rec=0
    true_non_rec=0
    false_non_rec=0

    cnt=0
    try_rec=0
    true_rec=0
    false_rec=0
    cum_acc=0
    # itarates through each session of one instalation_id
    for i, session in user_sample.groupby('game_session', sort=False):
        # i = game_session_id
        # session is a DataFrame that contain only one game_session
        
        # get some sessions information
        session_type = session['type'].iloc[0]
        session_title = session['title'].iloc[0]
        session_title_text = activities_labels[session_title]
                    
        rec=session.loc[session['event_code'].isin([4100,4110])]
        non_rec=session.loc[~(session['event_code'].isin([4100,4110]))]
        # for each assessment, and only this kind off session, the features below are processed
        # and a register are generated
        if (session_type == 'Assessment') & (test_set or len(session)>1):
            features={}
            # search for event_code 4100, that represents the assessments trial
            all_attempts = session.query(f'event_code == {win_code[session_title]}')
            # then, check the numbers of wins and the number of losses
            true_attempts = all_attempts['event_data'].str.contains('true').sum()
            false_attempts = all_attempts['event_data'].str.contains('false').sum()
            # copy a dict to use as feature template, it's initialized with some itens: 
            # {'Clip':0, 'Activity': 0, 'Assessment': 0, 'Game':0}
            features.update(event_code_count.copy())
            features.update(event_id_count.copy())
            features.update(title_count.copy())
            features.update(title_event_code_count.copy())
            features['true_rec_ass']=ass_true_rec
            features['false_rec_ass']=ass_false_rec
            features['tot_rec_ass']=ass_try_rec
    
      
            features['true_non_rec_ass']=ass_true_non_rec
            features['false_non_rec_ass']=ass_false_non_rec
            features['tot_non_rec_ass']=ass_try_non_rec
    
      
            features['true_rec_non_ass']=true_rec
            features['false_rec_non_ass']=false_rec
            features['tot_rec_non_ass']=try_rec
    
      
            features['true_non_rec']=true_non_rec
            features['false_non_rec']=false_non_rec
            features['tot_non_rec']=try_non_rec
            if cnt!=0:
                features['accuracy_cum']=cum_acc/cnt
            else:
                features['accuracy_cum']=0
            cnt+=1
            # get installation_id for aggregated features
            features['installation_id'] = session['installation_id'].iloc[-1]
            # add title as feature, remembering that title represents the name of the game
            features['session_title'] = session['title'].iloc[0]
            # the 4 lines below add the feature of the history of the trials of this player
            # this is based on the all time attempts so far, at the moment of this assessment
            
            accuracy = true_attempts/(true_attempts+false_attempts) if (true_attempts+false_attempts) != 0 else 0
            cum_acc+=accuracy
            # a feature of the current accuracy categorized
            # it is a counter of how many times this player was in each accuracy group
            if accuracy == 0:
                features['accuracy_group'] = 0
            elif accuracy == 1:
                features['accuracy_group'] = 3
            elif accuracy == 0.5:
                features['accuracy_group'] = 2
            else:
                features['accuracy_group'] = 1
            # mean of the all accuracy groups of this player
            true=rec.event_data.str.contains('true').sum()
            false=rec.event_data.str.contains('false').sum()
            tot=true+false
        
            # there are some conditions to allow this features to be inserted in the datasets
            # if it's a test set, all sessions belong to the final dataset
            # it it's a train, needs to be passed throught this clausule: session.query(f'event_code == {win_code[session_title]}')
            # that means, must exist an event_code 4100 or 4110
            if test_set:
                all_assessments.append(features)
            elif true_attempts+false_attempts > 0:
                
                #rec ass
                ass_true_rec+=true
                ass_false_rec+=false
                ass_try_rec=ass_false_rec+ass_true_rec
                
                
                all_assessments.append(features)
      
            else:
            #non rec ass
                ass_true_non_rec+=non_rec.event_data.str.contains('true').sum()
                ass_false_non_rec+=non_rec.event_data.str.contains('false').sum()
                ass_tot_non_rec=ass_true_non_rec+ass_false_non_rec
      
    
    
            
        
        # this piece counts how many actions was made in each event_code so far
        def update_counters(counter: dict, col: str):
                num_of_session_count = Counter(session[col])
                for k in num_of_session_count.keys():
                    x = k
                    if col == 'title':
                        x = activities_labels[k]
                    counter[x] += num_of_session_count[k]
                return counter
            
        event_code_count = update_counters(event_code_count, "event_code")
        event_id_count = update_counters(event_id_count, "event_id")
        title_count = update_counters(title_count, 'title')
        title_event_code_count = update_counters(title_event_code_count, 'title_event_code')
    
        #non rec
        true_non_rec+=(non_rec['event_data'].str.contains('true')).sum()
        false_non_rec+=(non_rec['event_data'].str.contains('false')).sum()
        tot_non_rec=true_non_rec+false_non_rec
        #rec
        true_rec+=(rec['event_data'].str.contains('true')).sum()
        false_rec+=(rec['event_data'].str.contains('false')).sum()
        tot_rec=false_rec+true_rec
        # counts how many actions the player has done so far, used in the feature of the same name
                        
    # if it't the test_set, only the last assessment must be predicted, the previous are scraped
    if test_set:
        return all_assessments[-1],all_assessments[:-1]
    # in the train_set, all assessments goes to the dataset
    return all_assessments
reduce_train, reduce_test, reduce_val= get_train_and_test(train, test)

In [ ]:
import lightgbm as lgb
# for df in [reduce_train,reduce_test,reduce_val]:
#     df=df
trn_ft=lgb.Dataset(reduce_train.select_dtypes(exclude=object).drop(['accuracy_group'],1),reduce_train['accuracy_group'])
val_ft=lgb.Dataset(reduce_val.select_dtypes(exclude=object).drop(['accuracy_group'],1),reduce_val['accuracy_group'])
mod=lgb.train( {'feature_fraction': 0.7975023410800904,'n_estimators':7297,'learning_rate':0.001,
  'max_depth': 14,
  'subsample': 0.7531064105216918},trn_ft)
pre=mod.predict(reduce_test.select_dtypes(exclude=object).drop(['accuracy_group'],1))
x,y,z=1.21519937, 1.70523477, 2.25945307
res=[]
for i in pre:
        if i<x:
            res.append(0)
        elif i<y:
            res.append(1)
        elif i<z:
            res.append(2)
        else:
            res.append(3)


sample_submission['accuracy_group']=res
sample_submission.to_csv('submission.csv', index=False)
sample_submission['accuracy_group'].value_counts()